In [1]:
def demandforecast(parameters):
    import tensorflow as tf
    import numpy as np
    import logging
    import pandas as pd
    import os
    import tensorflow as tf 
    from keras.models import Sequential, Model
    from keras.layers.convolutional import Conv1D, MaxPooling1D
    from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten, Dropout
    import keras
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import train_test_split

    
    logging.basicConfig(
        format="%(asctime)s %(levelname)-8s %(message)s",
        datefmt="%Y-%m-%dT%H:%M:%SZ",
        level=logging.INFO,
    )
    logging.info("--------------------------------------------------------------------------------------")
    logging.info(f"Input Parameters: {parameters}")
    logging.info("--------------------------------------------------------------------------------------\n\n")


    
    lr = float(parameters["lr"])
    num_epoch = int(parameters["num_epoch"])

    
    is_dist = parameters["is_dist"]
    num_workers = parameters["num_workers"]
    batch_size_per_worker = 64
    batch_size_global = batch_size_per_worker * num_workers
   
    class CustomCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            logging.info(
                "Epoch {}/{}. mse={:.4f} - loss={:.4f}".format(
                    epoch+1, num_epoch, logs["mse"], logs["loss"]
                )
            )
    
    sales_path='https://raw.githubusercontent.com/KantSagar/SampleData/master/sales_train.csv'
    items_path='https://raw.githubusercontent.com/KantSagar/SampleData/master/items.csv'
    resturant_path='https://raw.githubusercontent.com/KantSagar/SampleData/master/resturants.csv'
    
    df_sales = pd.read_csv(sales_path)
    df_items = pd.read_csv(items_path)
    df_resturant = pd.read_csv(resturant_path)
    
    
    def series_to_supervised(data, window=1, lag=1, dropnan=True):   
        cols, names = list(), list()
        # Input sequence (t-n, ... t-1)
        for i in range(window, 0, -1):
            cols.append(data.shift(i))
            names += [('%s(t-%d)' % (col, i)) for col in data.columns]
        # Current timestep (t=0)
        cols.append(data)
        names += [('%s(t)' % (col)) for col in data.columns]
        # Target timestep (t=lag)
        cols.append(data.shift(-lag))
        names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
        # Put it all together
        agg = pd.concat(cols, axis=1)
        agg.columns = names
        # Drop rows with NaN values
        if dropnan:
            agg.dropna(inplace=True)
        return agg
    df_items2 = df_items[['id','store_id']]
    df_train = df_sales.merge(df_items2,left_on='item_id',right_on='id')
    df_train[['date','item_id','item_count','store_id']]

    df_train = df_train.sort_values('date').groupby(['item_id', 'store_id', 'date'], as_index=False)
    df_train = df_train.agg({'item_count':['mean']})
    df_train.columns = ['item', 'store', 'date', 'sales']
    window = 29
    future_span = 30
    series = series_to_supervised(df_train.drop('date', axis=1), window=window, lag=future_span)
    # series.head()
    last_item = 'item(t-%d)' % window
    last_store = 'store(t-%d)' % window
    series = series[(series['store(t)'] == series[last_store])]
    series = series[(series['item(t)'] == series[last_item])]
    
    columns_to_drop = [('%s(t+%d)' % (col, future_span)) for col in ['item', 'store']]
    for i in range(window, 0, -1):
        columns_to_drop += [('%s(t-%d)' % (col, i)) for col in ['item', 'store']]
    series.drop(columns_to_drop, axis=1, inplace=True)
    series.drop(['item(t)', 'store(t)'], axis=1, inplace=True)
    
    # Label
    labels_col = 'sales(t+%d)' % future_span
    labels = series[labels_col]
    series = series.drop(labels_col, axis=1)

    X_train, X_valid, Y_train, Y_valid = train_test_split(series, labels.values, test_size=0.4, random_state=0)
    
    X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_valid_series = X_valid.values.reshape((X_valid.shape[0], X_valid.shape[1], 1))
    print('Train set shape', X_train_series.shape)
    print('Validation set shape', X_valid_series.shape)

    epochs = 500 
    batch = 256
    lr = 0.0003
    adam = tf.keras.optimizers.Adam(lr)

    model_cnn = Sequential()
    model_cnn.add(Conv1D(filters=64, kernel_size=8, activation='relu', input_shape=(X_train_series.shape[1], X_train_series.shape[2])))
    model_cnn.add(MaxPooling1D(pool_size=2))
    model_cnn.add(Flatten())
    model_cnn.add(Dense(50, activation='relu'))
    model_cnn.add(Dropout(0.2))
    model_cnn.add(Dense(1))
    model_cnn.compile(loss='mse', 
                      optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                      metrics=["mse"],)
    model_cnn.summary()
    
    cnn_history = model_cnn.fit(X_train_series, Y_train, callbacks=[CustomCallback()],
    validation_data=(X_valid_series, Y_valid), epochs=num_epoch, verbose=2)

    cnn_train_pred = model_cnn.predict(X_train_series)
    cnn_valid_pred = model_cnn.predict(X_valid_series)
    print('Train rmse:', np.sqrt(mean_squared_error(Y_train, cnn_train_pred)))
    print('Validation rmse:', np.sqrt(mean_squared_error(Y_valid, cnn_valid_pred)))

In [16]:
!pip install tensorflow --user

In [23]:
!pip install keras --user

In [2]:
# Set Parameters for Local Training.
parameters = {
    "lr": "0.01",
    "num_epoch": "20",
    "is_dist": False,
    "num_workers": 2
}

# Train Model locally in the Notebook.
demandforecast(parameters)

2023-12-12 07:11:07.049205: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-12T07:11:09Z INFO     --------------------------------------------------------------------------------------
2023-12-12T07:11:09Z INFO     Input Parameters: {'lr': '0.01', 'num_epoch': '20', 'is_dist': False, 'num_workers': 2}
2023-12-12T07:11:09Z INFO     --------------------------------------------------------------------------------------




Train set shape (64002, 30, 1)
Validation set shape (42668, 30, 1)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 23, 64)            576       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 11, 64)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 704)               0         
                                                                 
 dense (Dense)               (None, 50)                35250     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 

2023-12-12 07:11:11.050113: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (34)
2023-12-12 07:11:11.050161: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-cuda-0): /proc/driver/nvidia/version does not exist
2023-12-12 07:11:11.050628: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20


2023-12-12T07:11:15Z INFO     Epoch 1/20. mse=145.6462 - loss=145.6462


...

2001/2001 - 4s - loss: 103.6357 - mse: 103.6357 - val_loss: 78.0535 - val_mse: 78.0535 - 4s/epoch - 2ms/step
Epoch 19/20


2023-12-12T07:12:30Z INFO     Epoch 19/20. mse=102.0118 - loss=102.0118


2001/2001 - 4s - loss: 102.0118 - mse: 102.0118 - val_loss: 79.6638 - val_mse: 79.6638 - 4s/epoch - 2ms/step
Epoch 20/20


2023-12-12T07:12:35Z INFO     Epoch 20/20. mse=100.1161 - loss=100.1161


2001/2001 - 4s - loss: 100.1161 - mse: 100.1161 - val_loss: 78.1213 - val_mse: 78.1213 - 4s/epoch - 2ms/step
1334/1334 [==============================] - 1s 809us/step
Train rmse: 9.127654801748104
Validation rmse: 8.838623783583412


In [14]:
!kill 8005

In [1]:
# # importing the zipfile module 
# from zipfile import ZipFile 
  
# # loading the temp.zip and creating a zip object 
# with ZipFile("/home/jovyan/sales_train.csv.zip", 'r') as zObject: 
  
#     # Extracting all the members of the zip  
#     # into a specific location. 
#     zObject.extractall( 
#         path="/home/jovyan/")

In [5]:
from kubeflow import katib
parameters = {
    "lr": katib.search.double(min=0.01, max=0.4),
    "num_epoch": katib.search.int(min=10, max=500),
    "is_dist": False,
    "num_workers": 1
}
exp_name = "forecast-cmaes-hpt2"
katib_client = katib.KatibClient()

katib_client.tune(
    name=exp_name,
    objective=demandforecast,
    parameters=parameters,
    base_image='docker.io/amaksimov/python_data_science:latest',
    algorithm_name="cmaes",
    objective_metric_name="mse",
    additional_metric_names=["loss"],
    max_trial_count=100,
    parallel_trial_count=2,
)

Experiment sagar-kant/forecast-cmaes-hpt2 has been created


In [1]:
# !pip install git+https://github.com/kubeflow/katib.git#subdirectory=sdk/python/v1beta1
# !pip install git+https://github.com/kubeflow/training-operator.git#subdirectory=sdk/python

In [16]:
status = katib_client.is_experiment_succeeded(exp_name)
print(f"Katib Experiment is Succeeded: {status}\n")

best_hps = katib_client.get_optimal_hyperparameters(exp_name)

if best_hps != None:
    print("Current Optimal Trial\n")
    print(best_hps)
    
    for hp in best_hps.parameter_assignments:
        if hp.name == "lr":
            best_lr = hp.value
        else:
            best_num_epoch = hp.value

Katib Experiment is Succeeded: False



In [18]:
from kubeflow import katib

In [4]:
!pip install kubeflow-katib

  Obtaining dependency information for kubeflow-katib from https://files.pythonhosted.org/packages/19/c4/28ced11972001ef18b4b9776ffb71940b89d36229ab7408c16b388f65379/kubeflow_katib-0.16.0-py3-none-any.whl.metadata
  Using cached kubeflow_katib-0.16.0-py3-none-any.whl.metadata (1.4 kB)
  Obtaining dependency information for kubernetes>=23.6.0 from https://files.pythonhosted.org/packages/f5/6a/1f69c2d8b1ff03f8d8e10d801f4ac3016ed4c1b00aa9795732c6ec900bba/kubernetes-28.1.0-py2.py3-none-any.whl.metadata
  Using cached kubernetes-28.1.0-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached kubeflow_katib-0.16.0-py3-none-any.whl (107 kB)
Using cached kubernetes-28.1.0-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: kubernetes
    Found existing installation: kubernetes 12.0.1
    Uninstalling kubernetes-12.0.1:
      Successfully uninstalled kubernetes-12.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the so